<a href="https://colab.research.google.com/github/Harrisson3/Breast-Cancer-Dataset/blob/main/ashionMnistConvnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
## Fashion-MNIST vs. MNIST Image Recognition

This notebook compares the performance of a convolutional neural network (CNN) on the MNIST and Fashion-MNIST datasets.

### Datasets:
- **MNIST**: 28x28 grayscale images of handwritten digits (0-9).
- **Fashion-MNIST**: 28x28 grayscale images of clothing articles, labeled in 10 categories:
  - 0: T-shirt/top
  - 1: Trouser
  - 2: Pullover
  - 3: Dress
  - 4: Coat
  - 5: Sandal
  - 6: Shirt
  - 7: Sneaker
  - 8: Bag
  - 9: Ankle boot
- Each dataset contains 60,000 training samples and 10,000 test samples.

### Tasks:
1. Train a convnet on MNIST and Fashion-MNIST and compare performance.
2. Evaluate training times.
3. Add a Dense layer with 4096 neurons and analyze its impact.
"""

import tensorflow as tf
from tensorflow import keras
import numpy as np
import time

# Load MNIST and Fashion-MNIST datasets
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = keras.datasets.mnist.load_data()
(x_train_fashion, y_train_fashion), (x_test_fashion, y_test_fashion) = keras.datasets.fashion_mnist.load_data()

# Normalize data
x_train_mnist, x_test_mnist = x_train_mnist / 255.0, x_test_mnist / 255.0
x_train_fashion, x_test_fashion = x_train_fashion / 255.0, x_test_fashion / 255.0

# Expand dimensions for channels
x_train_mnist = np.expand_dims(x_train_mnist, axis=-1)
x_test_mnist = np.expand_dims(x_test_mnist, axis=-1)
x_train_fashion = np.expand_dims(x_train_fashion, axis=-1)
x_test_fashion = np.expand_dims(x_test_fashion, axis=-1)

# Define a function to create the model
def create_convnet(add_dense_layer=False):
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.Flatten(),
    ])

    if add_dense_layer:
        model.add(keras.layers.Dense(4096, activation='relu'))

    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Train and evaluate models
def train_and_evaluate(dataset_name, x_train, y_train, x_test, y_test, add_dense_layer=False):
    print(f"Training on {dataset_name} with{'out' if not add_dense_layer else ''} additional Dense layer...")
    model = create_convnet(add_dense_layer)
    start_time = time.time()
    model.fit(x_train, y_train, epochs=5, batch_size=64, verbose=1, validation_split=0.1)
    training_time = time.time() - start_time
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"Test accuracy on {dataset_name}: {test_acc:.4f}, Training time: {training_time:.2f} sec\n")
    return test_acc, training_time

# Train models on both datasets
results = {}
for dataset_name, x_train, y_train, x_test, y_test in zip(
    ["MNIST", "Fashion-MNIST"],
    [x_train_mnist, x_train_fashion],
    [y_train_mnist, y_train_fashion],
    [x_test_mnist, x_test_fashion],
    [y_test_mnist, y_test_fashion]
):
    results[dataset_name] = train_and_evaluate(dataset_name, x_train, y_train, x_test, y_test)
    results[dataset_name + " + Dense 4096"] = train_and_evaluate(dataset_name, x_train, y_train, x_test, y_test, add_dense_layer=True)

# Print summary
for key, (acc, time) in results.items():
    print(f"{key}: Accuracy = {acc:.4f}, Training Time = {time:.2f} sec")


844/844 ━━━━━━━━━━━━━━━━━━━━ 50s 55ms/step - accuracy: 0.8670 - loss: 0.4406 - val_accuracy: 0.9845 - val_loss: 0.0529
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 83s 56ms/step - accuracy: 0.9822 - loss: 0.0566 - val_accuracy: 0.9877 - val_loss: 0.0399
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 46s 54ms/step - accuracy: 0.9887 - loss: 0.0349 - val_accuracy: 0.9892 - val_loss: 0.0359
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 81s 53ms/step - accuracy: 0.9913 - loss: 0.0272 - val_accuracy: 0.9907 - val_loss: 0.0324
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 84s 55ms/step - accuracy: 0.9936 - loss: 0.0217 - val_accuracy: 0.9908 - val_loss: 0.0374
Test accuracy on MNIST: 0.9892, Training time: 379.68 sec

Training on MNIST with additional Dense layer...
Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 93s 107ms/step - accuracy: 0.8942 - loss: 0.3267 - val_accuracy: 0.9807 - val_loss: 0.0616
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 90s 106ms/step - accuracy: 0.9845 - loss: 0.0515 - val_accuracy: 0.9902 - val_loss: 0.03